In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd
from pathlib import Path
import re

DATA_DIR = Path("/content/drive/MyDrive/NBA player stats 2000-2023")

def sort_key(path):
    m = re.search(r"\((\d+)\)", path.name)
    return int(m.group(1)) if m else 0   # unnumbered file = 0

xls_files = sorted(DATA_DIR.glob("sportsref_download*.xls"), key=sort_key)

print("Files in order that will be used for years 2000..2023:\n")
for i, f in enumerate(xls_files):
    print(f"{2000 + i}  <--  {f.name}")


print("\nTotal files:", len(xls_files))



Files in order that will be used for years 2000..2023:

2000  <--  sportsref_download.xls
2001  <--  sportsref_download (1).xls
2002  <--  sportsref_download (2).xls
2003  <--  sportsref_download (3).xls
2004  <--  sportsref_download (4).xls
2005  <--  sportsref_download (5).xls
2006  <--  sportsref_download (6).xls
2007  <--  sportsref_download (7).xls
2008  <--  sportsref_download (8).xls
2009  <--  sportsref_download (9).xls
2010  <--  sportsref_download (10).xls
2011  <--  sportsref_download (11).xls
2012  <--  sportsref_download (12).xls
2013  <--  sportsref_download (13).xls
2014  <--  sportsref_download (14).xls
2015  <--  sportsref_download (15).xls
2016  <--  sportsref_download (16).xls
2017  <--  sportsref_download (17).xls
2018  <--  sportsref_download (18).xls
2019  <--  sportsref_download (19).xls
2020  <--  sportsref_download (20).xls
2021  <--  sportsref_download (21).xls
2022  <--  sportsref_download (22).xls
2023  <--  sportsref_download (23).xls
2024  <--  sportsref_d

In [ ]:
import pandas as pd
from pathlib import Path

DATA_DIR = Path("/content/drive/MyDrive/NBA player stats 2000-2023")


df_2000_2023 = pd.read_csv(DATA_DIR / "nba_player_stats_2000_2023_fixed_years.csv")
print("Loaded 2000–2023 rows:", len(df_2000_2023))


# LOAD RAW 2024 & 2025 (NO CLEANING YET)

def load_xls(file, year):
    temp = pd.read_html(str(DATA_DIR / file))[0]
    temp = temp[temp["Player"] != "Player"]   # remove header rows
    temp["Year"] = year
    return temp

df_2024 = load_xls("sportsref_download (24).xls", 2024)
df_2025 = load_xls("sportsref_download (25).xls", 2025)

df_new = pd.concat([df_2024, df_2025], ignore_index=True)
print("Loaded 2024–2025 rows:", len(df_new))



# APPLY 2TM CLEANING ONLY TO 2024 & 2025

def clean_only_total_rows(df):
    team_col = "Tm" if "Tm" in df.columns else "Team"
    total_tags = ["2TM", "3TM", "4TM", "5TM"]

    cleaned = []

    for year in df["Year"].unique():
        chunk = df[df["Year"] == year].copy()

        # only clean 2024 + 2025
        if year not in [2024, 2025]:
            cleaned.append(chunk)
            continue

        # player-year pairs with TOTAL rows
        totals = chunk[chunk[team_col].isin(total_tags)][["Player", "Year"]].drop_duplicates()
        total_pairs = set(map(tuple, totals.values))

        # keep everything if no total row exists for that player
        def keep(row):
            pair = (row["Player"], row["Year"])
            if pair not in total_pairs:
                return True
            return row[team_col] in total_tags  # keep only TOTAL rows

        cleaned.append(chunk[chunk.apply(keep, axis=1)])

    return pd.concat(cleaned, ignore_index=True)


df_new_clean = clean_only_total_rows(df_new)
print("Rows after cleaning 2024–2025:", len(df_new_clean))



# 4️⃣ FINAL MERGE

df_final = pd.concat([df_2000_2023, df_new_clean], ignore_index=True)

df_final = df_final.sort_values(["Year", "Player"]).reset_index(drop=True)

OUTPUT = DATA_DIR / "nba_player_stats_2000_2025_FINAL.csv"
df_final.to_csv(OUTPUT, index=False)

print("\n✔ SAVED:", OUTPUT)
print("FINAL ROW COUNT:", len(df_final))
print("YEARS:", sorted(df_final["Year"].unique()))



Loaded 2000–2023 rows: 11981
Loaded 2024–2025 rows: 1000
Rows after cleaning 2024–2025: 772

✔ SAVED: /content/drive/MyDrive/NBA player stats 2000-2023/nba_player_stats_2000_2025_FINAL.csv
FINAL ROW COUNT: 12753
YEARS: [np.int64(2000), np.int64(2001), np.int64(2002), np.int64(2003), np.int64(2004), np.int64(2005), np.int64(2006), np.int64(2007), np.int64(2008), np.int64(2009), np.int64(2010), np.int64(2011), np.int64(2012), np.int64(2013), np.int64(2014), np.int64(2015), np.int64(2016), np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023), np.int64(2024), np.int64(2025)]


In [ ]:
import pandas as pd
from pathlib import Path

DATA_DIR = Path("/content/drive/MyDrive/NBA player stats 2000-2023")
FILE = DATA_DIR / "nba_player_stats_2000_2025_FINAL.csv"
df = pd.read_csv(FILE)

# Ensure Rk is numeric
df["Rk"] = pd.to_numeric(df["Rk"], errors="coerce")

# Sort within each year
df_sorted = df.sort_values(["Year", "Rk"]).reset_index(drop=True)

# Save fixed file
OUTPUT = DATA_DIR / "nba_player_stats_2000_2025_sorted_by_rk.csv"
df_sorted.to_csv(OUTPUT, index=False)

print("✔ Saved sorted dataset:", OUTPUT)
print("Years:", sorted(df_sorted["Year"].unique()))


✔ Saved sorted dataset: /content/drive/MyDrive/NBA player stats 2000-2023/nba_player_stats_2000_2025_sorted_by_rk.csv
Years: [np.int64(2000), np.int64(2001), np.int64(2002), np.int64(2003), np.int64(2004), np.int64(2005), np.int64(2006), np.int64(2007), np.int64(2008), np.int64(2009), np.int64(2010), np.int64(2011), np.int64(2012), np.int64(2013), np.int64(2014), np.int64(2015), np.int64(2016), np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023), np.int64(2024), np.int64(2025)]


In [ ]:
import pandas as pd
from pathlib import Path

# 📂 paths
DATA_DIR = Path("/content/drive/MyDrive/NBA player stats 2000-2023")
INPUT  = DATA_DIR / "nba_player_stats_2000_2025_sorted_by_rk.csv"     # change if your file name is different
OUTPUT = DATA_DIR / "nba_player_stats_2000_2025_clean"

# 1) Load data
df = pd.read_csv(INPUT)
print("Original rows:", len(df))

# 2) Detect team column and multi-team tags
team_col   = "Tm" if "Tm" in df.columns else "Team"
multi_tags = ["2TM", "3TM", "4TM", "5TM", "6TM"]

# 3) Find (Player, Year) combos that have a 2TM–6TM row
has_multi = (
    df[df[team_col].isin(multi_tags)][["Player", "Year"]]
      .drop_duplicates()
      .assign(has_multi=True)
)

# 4) Merge the flag back onto the main dataframe
df_flagged = df.merge(has_multi, on=["Player", "Year"], how="left")

# 5) Build keep mask:
#    - if has_multi is True: keep ONLY rows where team is 2TM–6TM
#    - if has_multi is NaN: keep ALL rows (no multi-team total exists)
keep_mask = (
    df_flagged["has_multi"].isna() |
    (df_flagged[team_col].isin(multi_tags))
)

dedup_df = df_flagged[keep_mask].drop(columns=["has_multi"])
print("After 2TM–6TM dedup rows:", len(dedup_df))

# 6) Sort by Year, then Rk (lowest Rk first in each year)
if "Rk" in dedup_df.columns:
    dedup_df["Rk"] = pd.to_numeric(dedup_df["Rk"], errors="coerce")
    dedup_df = dedup_df.sort_values(["Year", "Rk"])
else:
    dedup_df = dedup_df.sort_values(["Year", "Player"])

dedup_df = dedup_df.reset_index(drop=True)

# 7) Save result
dedup_df.to_csv(OUTPUT, index=False)
print("\n✔ Saved cleaned, sorted file to:\n", OUTPUT)
print("Years present:", sorted(dedup_df["Year"].unique()))
print("Final rows:", len(dedup_df))


Original rows: 12753
After 2TM–6TM dedup rows: 10426

✔ Saved cleaned, sorted file to:
 /content/drive/MyDrive/NBA player stats 2000-2023/nba_player_stats_2000_2025_clean
Years present: [np.int64(2000), np.int64(2001), np.int64(2002), np.int64(2003), np.int64(2004), np.int64(2005), np.int64(2006), np.int64(2007), np.int64(2008), np.int64(2009), np.int64(2010), np.int64(2011), np.int64(2012), np.int64(2013), np.int64(2014), np.int64(2015), np.int64(2016), np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023), np.int64(2024), np.int64(2025)]
Final rows: 10426
